# AI Overlord
## Descent: Journeys in the Dark AI Uprising

In [2]:
############### IMPORTS ###############

import numpy as np
import pandas as pd
from descent_ai import *

############### FORMATTING ###############

# jupyter notebook full-width display
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_colwidth', 1000)


In [14]:
# import moster dataframe
df = MONSTER_DF.copy()
df.name = df.name.str.replace(' Master', '').str.replace(' ', '_').str.lower()
df = pd.concat([
    df.loc[df['rank'] != 'master', ['name', 'expansion', 'abilities', 'attack_type']].reset_index(drop=True), 
    df.groupby('name').sum('quantity')[['quantity']].reset_index()
    ], axis=1
).iloc[:, [0, 3, 5, 1, 2]]
df

,name,attack_type,quantity,expansion,abilities
0,bane_spider,magic,9,base,Poison
1,beastman,melee,9,base,+1 Damage
2,blood_ape,melee,6,AoD,Leap
3,chaos_beast,unknown,3,AoD,"Morph, Sorcery 1"
4,dark_priest,magic,6,AoD,Dark Prayer
5,deep_elf,melee,3,AoD,"Shadowcloak, Pierce 3"
6,demon,magic,2,base,"Aura 1, Sorcery 2, Fear 1"
7,dragon,ranged,2,base,"Burn, Breath, Pierce 5"
8,ferrox,melee,6,WoD,Bleed
9,giant,melee,2,base,"Reach, Stun"


In [14]:
# example monsters / encounters
m = Monsters(MONSTER_DF)
print('Monsters')
print(m.quest_monsters)
print(m.quest_boss, 'Boss')
print()
print('Encounters')
for encounter in m.encounters.values():
    print(encounter)

Monsters
['Beastman', 'Lava Beetle', 'Troll']
Chaos Beast Boss

Encounters
['Beastman', 'Beastman', 'Beastman Master', 'Lava Beetle']
['Beastman', 'Beastman', 'Beastman', 'Beastman', 'Lava Beetle', 'Troll', 'Troll']
['Beastman', 'Beastman Master', 'Beastman Master', 'Troll Master', 'Troll Master']
['Beastman', 'Troll', 'Chaos Beast Boss']


In [ ]:
class Archetypes:
    def __init__(self, dataframe):
        self._DATA = dataframe.copy()

    def get_archetype(self, monster_name):
        pass


class MonsterAI:
    ARCHETYPES = Archetypes(MONSTER_DF)

    def __init__(self, monster_name):
        self.monster_name = monster_name
        self.archtype = self.get_archetypes(monster_name)

    def get_archetype(self, monster_name):
        return ARCHETYPES.get_archetype(monster_name)      

    def target(self):
        pass

    def movement(self):
        pass

    def special_triggers(self):
        # eg if less than half health, do X
        pass

    def special_powers(self):
        # eg minion = spawn 2x, all have 1HP
        pass

    def boss_powers(self):
        pass

# OLD METHOD, NOT GREAT

In [2]:
############### MONSTER STRATEGY ###############

def monster_order(chance_heroes_choose=0.1):

    monster_order = {
        'Monster Order by Type:': ['melee', 'ranged'],
        'Monster Order by Rank:': ['masters', 'minion']
    }
    
    dataframe = pd.DataFrame()
    for key in monster_order.keys():
        order = ''
        for item in np.random.permutation(monster_order[key]):
            if order:
                order += ' => ' + item.upper()
            else:
                order += item.upper()
        
        if np.random.random() < chance_heroes_choose: 
            order = 'HEROES CHOOSE'
        
        dataframe.loc[key, ''] = order

    return dataframe


# # formatted test output
# print('\n==================================\nMONSTER ORDER OF ATTACK')
# display(monster_order(0.5).style.set_properties(**{'text-align': 'left'}))
# print('\n')

In [3]:
############### OVERLORD CARDS ###############

def overlord_card_rule(draw_card_manually=True, max_cards=5):
    
    # overlord card rules
    overlord_rules = {
        'What to do with the overlord card': [
            'ignore and discard', 
            'play first time possible, discard', 
            'play first time possible, discard, draw a new card, repeat',
            'when possible, roll red, play the card if you roll a Surge, then keep card and repeat for the round',
            'when possible, roll red, play the card if you roll a Surge, then discard the card'
        ]
    }
    
    card_rule = np.random.choice(overlord_rules['What to do with the overlord card'])
    print(card_rule)
    
    if draw_card_manually or 'ignore and discard' in card_rule:
        pass
    else:
        if 'draw a new card' not in card_rule:  # only draw more than one card if max cards
            min_cards, max_cards = 1, 1
        else:
            print('\n...once you run out of cards, you are done')
            min_cards = 2
        
        display(overlord_card(min_cards=min_cards, max_cards=max_cards))


def overlord_card(min_cards=1, max_cards=1):
    
    if min_cards > max_cards:
        max_cards = min_cards
    
    # choose random card
    overlord_cards = {
        'Pit Trap': 'Play this card when a hero enters an empty space. He tests Awareness. If he fails, he suffers 1 Heart and loses 1 movement point. If he has no movement points to lose (such as if he suffered fatigue to move), he is Stunned.',
        'Critical Blow': 'Play this card when a monster attacks a hero, after rolling dice. \nThe attack gains: \nSurge: +3 Heart',
        'Poison Dart': 'Play this card when a hero opens a door or searches. He tests Awareness or Might (your choice). If he passes, draw 1 Overlord Card. If he fails, he suffers 1 Heart, 1 Fatigue, and he is Poisoned.',
        'Word of Misery': 'Play this card at the start of your turn. During this turn, each time a hero suffers any Heart, he also suffers 1 Fatigue in addition to the Heart suffered.',
        'Dark Charm': 'Play this card on a hero at the start of your turn. The hero tests Willpower. If he passes, draw 1 Overlord Card. If he fails, you may perform a move or attack action with that hero as if he were one of your monsters this turn. You cannot force him to suffer Fatigue or use a Potion, but you may force him to attack himself.',
        'Dark Might': 'Play this card after you roll dice for an attack. Add 1 Surge to the results.',
        'Tripwire': 'Play this card when a hero enters an empty space during a move action. He tests Awareness. If he fails, he must end his move action (he can still suffer Fatigue to move further, or perform a second move action if this was his first action).',
        'Dash': 'Play this card when activating a monster during your turn. That monster may perform an additional move action this turn in addition to its normal 2 actions.',
        'Frenzy': 'Play this card when activating a monster during your turn. That monster may perform an additional attack action this turn in addition to its normal 2 actions.',
        'Dark Fortune': 'Play this card after you roll dice. You may reroll 1 die.'
    }
    return pd.DataFrame(overlord_cards, index=['']).T.sample(np.random.randint(1, max_cards+1)).style.set_properties(**{'text-align': 'left'})


# # formatted test output
# print('\n==================================\nOVERLORD CARDS\n\ndraw a card...')
# overlord_card_rule(False)  # false means the computer draws the cards
# print('\n')

In [65]:
############### MONSTER TACTICS ###############

# choose 1 random output from lists
melee = {
    'Quest:': ['top priority', 'attack but attempt', 'ignore'],
    'Target:': ['closest', 'most damaged in range', 'most surroundable'], 
    'Move:': ['attack then back off', 'attack and stay adjacent'],
    'Special:': [
        'use special instead of attack (both if possible)', 
        'attack then special if possible', 
        'no special ability this turn'
    ],
}

# choose 1 random output from lists
ranged = {
    'Quest:': ['top priority', 'attack but attempt', 'ignore'],
    'Target:': ['closest', 'most damaged in range', 'most surroundable'],
    'Range:': ['within range 3', 'within roll-2', 'within roll-1'],
    'Move:': ['attack then back off', 'attack and stay at range'],
    'Special:': [
        'use special instead of attack (both if possible)', 
        'attack then special if possible', 
        'no special ability this turn'
    ],
}

def attack(attacker_dict):

    dataframe = pd.DataFrame()

    for key in attacker_dict.keys():
        dataframe.loc[key, ''] = np.random.choice(attacker_dict[key])

    return dataframe


# # formatted test output
# print('\n\n==================================\nMELEE ATTACKERS')
# display(attack(melee))
# print('\n\n==================================\nRANGE ATTACKERS')
# display(attack(ranged))
# print('\n')

In [5]:
############### SUMMARY OF TURN ###############

def turn_rules_summary(overlord_cards=False, draw_card_manually=True, max_cards=5, chance_heroes_choose=0.1):
    print('\n')
    print('------------------------------ MONSTER STRATEGY ------------------------------')
    order = monster_order(chance_heroes_choose)
    display(order.style.set_properties(**{'text-align': 'left'}))
    print('\n')
    if overlord_cards:
        print('------------------------------ OVERLORD CARDS ------------------------------')
        print('\ndraw a card...')
        overlord_card_rule(draw_card_manually, max_cards)  # false means the computer draws the cards
        print('\n')
    print('------------------------------ MONSTER TACTICS ------------------------------')
    if order.iloc[0].str.split()[0][0].lower() == 'melee':  # same order as order above
        print('\n------------------------------\nMELEE ATTACKERS')
        display(attack(melee))
        print('\n------------------------------\nRANGED ATTACKERS')
        display(attack(ranged))
    else:  # this includes 'heroes choose'
        print('\n------------------------------\nRANGED ATTACKERS')
        display(attack(ranged))
        print('\n------------------------------\nMELEE ATTACKERS')
        display(attack(melee))
    print('-------------------------------------------------------------------------------')

# # test
# turn_rules_summary()

In [6]:
############### GAME LOOP ###############

def game_loop(round_counter=True, overlord_cards=False, draw_card_manually=True, max_cards=5, chance_heroes_choose=0.1):
    round_count=0
    while True:
        round_count += 1
        if round_counter:
            print('##############################################################################')
            print(f'                                      ROUND {round_count}')
            print('##############################################################################')
        turn_rules_summary(overlord_cards, draw_card_manually, max_cards, chance_heroes_choose)
        # end game condition
        print('\n\nPRESS ENTER TO CONTINUE...\n\n\n\n\n')
#         time.sleep(0.3)  # just to avoid the input printing above the end of the stuff before it in jupyter
        if(input() != ''):  # anything other than enter exits (but only after an enter)
            break


# # test
# game_loop(True, True, True, 5, .5)

In [34]:
############### SUMMARY OF TURN ###############
# ALT EVEN LESS SPACE

def turn_rules_summary(overlord_cards=False, draw_card_manually=True, max_cards=5, chance_heroes_choose=0.1):
    print('\n-------------------------------------------------------------------------------')
    if overlord_cards:
        print('\ndraw a card...')
        overlord_card_rule(draw_card_manually, max_cards)  # false means the computer draws the cards
        
    order = monster_order(chance_heroes_choose)
    print(order.to_string())

    if order.iloc[0].str.split()[0][0].lower() == 'melee':  # same order as order above
        print('\nMELEE ATTACKERS')
        print(attack(melee).to_string())
        print('\nRANGED ATTACKERS')
        print(attack(ranged).to_string())
    else:  # this includes 'heroes choose'
        print('\nRANGED ATTACKERS')
        print(attack(ranged).to_string())
        print('\nMELEE ATTACKERS')
        print(attack(melee).to_string())
    

# # test
# turn_rules_summary(True)

In [69]:
# GAME
turn_rules_summary(True, True, 5, .5)


-------------------------------------------------------------------------------

draw a card...
when possible, roll red, play the card if you roll a Surge, then discard the card
                                     
Monster Order by Type:  HEROES CHOOSE
Monster Order by Rank:  HEROES CHOOSE

RANGED ATTACKERS
                                                          
Quest:                                  attack but attempt
Target:                                  most surroundable
Range:                                       within roll-1
Move:                             attack and stay at range
Special:  use special instead of attack (both if possible)

MELEE ATTACKERS
                                                          
Quest:                                        top priority
Target:                              most damaged in range
Move:                             attack and stay adjacent
Special:  use special instead of attack (both if possible)
